In [1]:
! python3 ./test_icp.py

(11690, 3)
2025-02-16 14:20:01.692 Python[16396:1054595] +[IMKClient subclass]: chose IMKClient_Legacy
2025-02-16 14:20:01.692 Python[16396:1054595] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
(11824, 3)
(8569, 3)
(5032, 3)


In [1]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
from scipy.spatial import cKDTree
from utils import read_canonical_model, load_pc, visualize_icp_result

def rotationZ(angle):
    return np.array([[np.cos(angle), -np.sin(angle), 0],
                     [np.sin(angle), np.cos(angle), 0],
                     [0, 0, 1]])

def remap(target, target_prox):
    mapping = np.arange(target_prox.shape[0])
    # build KD tree for quicker search
    target_KDtree = cKDTree(target)
    for i in range(target_prox.shape[0]):
        _, mapping[i] = target_KDtree.query(target_prox[i])

    return mapping

def ICP(source, target, iters):
    mean_source = np.mean(source, axis=0)
    delta_source = source - mean_source
    mean_target = np.mean(target, axis=0)
    delta_target = target - mean_target
    # try to initialize different rotation matrix
    rotation_matrix = rotationZ(0)
    p = np.zeros(3)

    for iter in range(iters):
        # map to closest point
        target_prox = (rotation_matrix.T @ (source - p).T).T
        mapping = remap(target, target_prox)

        # calculate dist of point cloud
        dist = 0
        for i in range(source.shape[0]):
            dist = dist + np.linalg.norm(target[mapping[i]] - target_prox[i])
        print(f"iter: {iter}, dist = {dist}")

        # iterate next rotation matrix and p
        Q = np.zeros((3,3))
        for j in range(source.shape[0]):
            Q = Q + delta_source[j].reshape((3, 1)) @ delta_target[mapping[j]].reshape((1, 3))
        U, _, V = np.linalg.svd(Q)
        rotation_matrix = U @ V
        if np.linalg.det(rotation_matrix) < 0.0:
            V[-1, :] *= -1
            rotation_matrix = U @ V
        p = mean_source - (rotation_matrix @ mean_target)

    pose = np.eye(4)
    pose[:3,:3] = rotation_matrix.T
    pose[:3, -1] = -(rotation_matrix.T @ p)
    return pose

obj_name = 'drill' # drill or liq_container
# num_pc = 4 # number of point clouds
num_pc = 1 # number of point clouds

source_pc = read_canonical_model(obj_name)
                    
for i in range(num_pc):
    # estimated_pose, you need to estimate the pose with ICP
    target_pc = load_pc(obj_name, i)
    pose = ICP(source_pc, target_pc, 20)
    print(pose)

# visualize the estimated result
visualize_icp_result(source_pc, target_pc, pose)



iter: 0, dist = 24245.73628370118
iter: 1, dist = 1765.7812370194565
iter: 2, dist = 1717.303290170807
iter: 3, dist = 1684.8245423854728
iter: 4, dist = 1657.9172332600617
iter: 5, dist = 1631.1356826457363
iter: 6, dist = 1604.7284607704798
iter: 7, dist = 1579.3045292159948
iter: 8, dist = 1556.492576854424
iter: 9, dist = 1536.5916682790203
iter: 10, dist = 1520.177026633927
iter: 11, dist = 1507.1884924353953
iter: 12, dist = 1497.430958162264
iter: 13, dist = 1490.715109764071
iter: 14, dist = 1486.2199357060672
iter: 15, dist = 1483.1379185080962
iter: 16, dist = 1481.0450506485604
iter: 17, dist = 1479.5725435332456
iter: 18, dist = 1478.510620965791
iter: 19, dist = 1477.8875040210826
[[ 0.41145474  0.83239535 -0.37124518  0.5701403 ]
 [-0.90513041  0.42098614 -0.05924196 -0.03810166]
 [ 0.10697634  0.36040068  0.92664309 -0.0290082 ]
 [ 0.          0.          0.          1.        ]]


2025-02-16 16:51:12.259 Python[18307:1181836] +[IMKClient subclass]: chose IMKClient_Legacy
2025-02-16 16:51:12.259 Python[18307:1181836] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
